In [1]:
from proc_handler import SeqClassHandler
import torch.nn as nn
import torch.nn.functional as F
from fastai.text import *
from pathlib import Path
import pandas as pd
import apex.optimizers.fused_lamb as LAMB
from functools import partial

In [2]:
seed = 42
bs = 20

In [3]:
# Initialise Processors (Tokenize and create Vocab)
model_name = 'distilbert-base-uncased'
processor = SeqClassHandler(model_name)

In [4]:
# repr
processor

Model: distilbert-base-uncased
Original Tokenizer: <transformers.tokenization_distilbert.DistilBertTokenizer object at 0x7fb4c075aed0>

In [5]:
# Split train/valid set from df
path = Path('./data')
data_name = 'IMDB Dataset.csv'
base_csv = pd.read_csv(path/data_name)
base_csv.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
# TextList and friends (TextDataBunch) require padding token id in the tokenizer vocabulary
tok_pad_idx = processor.pt_tokenizer.pad_token_id

In [7]:
# Load databunch from our DataFrame
data =  (TextList.from_df(base_csv, cols='review',processor=processor())
        .split_by_rand_pct(0.2,seed = seed)
        .label_from_df(cols= 'sentiment')
        .databunch(bs = bs, pad_first = False, pad_idx=tok_pad_idx))

In [8]:
data.train_ds[0]

(Text [CLS] one of the other reviewers has mentioned that after watching just 1 oz episode you ' ll be hooked . they are right , as this is exactly what happened with me . < br / > < br / > the first thing that struck me about oz was its brutality and un ##fl ##in ##ching scenes of violence , which set in right from the word go . trust me , this is not a show for the faint hearted or tim ##id . this show pulls no punches with regards to drugs , sex or violence . its is hardcore , in the classic use of the word . < br / > < br / > it is called oz as that is the nickname given to the oswald maximum security state pen ##ite ##nta ##ry . it focuses mainly on emerald city , an experimental section of the prison where all the cells have glass fronts and face inward ##s , so privacy is not high on the agenda . em city is home to many . . aryan ##s , muslims , gangs ##tas , latino ##s , christians , italians , irish and more . . . . so sc ##uf ##fles , death stares , dod ##gy dealings and shad

In [9]:
processor.model

CustTransformerModel(
  (pt_model): DistilBertForSequenceClassification(
    (distilbert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (dropout): Dropout(p=0.1, inplace=False)
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((